# RAG

Retrieval-Augmented Generation (RAG) é uma técnica que combina modelos de linguagem com mecanismos de recuperação de informações para melhorar a geração de texto.

O RAG revolucionou a interação, compreensão e geração de linguagem humana pelos sistemas de IA. Tornou os modelos de linguagem mais versáteis e inteligentes, sendo crucial para chatbots sofisticados e ferramentas complexas de criação de conteúdo.
Uma das aplicações mais poderosas dos LLMs é a criação de chatbots sofisticados de perguntas e respostas (Q&A). Esses chatbots podem responder a perguntas sobre informações específicas usando RAG.

> [ ver slides para mais sobre RAG ]

O LangChain tem vários componentes projetados para ajudar a criar aplicativos de perguntas e respostas e aplicativos RAG de forma mais geral.

## Importações

In [ ]:
import torch
from langchain_groq import ChatGroq

from langchain.prompts import PromptTemplate

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Carregando a LLM


In [ ]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

## Template e Chain

In [ ]:
# LLAMA 3
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

template

In [ ]:
prompt = PromptTemplate.from_template(template)
prompt

> Definindo o nosso primeiro exemplo

Como primeiro exemplo, vamos usar o dia de hoje. Aqui é algo onde o contexto através de RAG pode ser útil já que está totalmente fora do alcance da LLM retornar o dia hoje, é algo que ela simplesmente não tem como saber sozinha

In [ ]:
chain = prompt | llm

chain.invoke({"pergunta": "Que dia é hoje?"})


## Prompt para RAG

Para implementar o RAG, devemos reservar um espaço no template do prompt para que seja alocado nessa parte o contexto que queremos usar

Basearemos nesse template que está hospedado no Hub do LangSmith https://smith.langchain.com/hub/rlm/rag-prompt É um prompt bastante usado para esse objetivo

Mais tarde ensinaremos como puxar diretamente os prompts de lá sem precisar copiar e colar. Mas agora, fazemos assim pois queremos adequar ao template do modelo Llama 3

In [ ]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

## Definindo o contexto

Agora vamos usar uma função do python que nos retorna o dia


In [ ]:
from datetime import date

dia = date.today()
print(dia)

In [ ]:
contexto = "Você sabe que hoje é dia '{}'".format(dia)
print(contexto)

## Criação da Chain / Geração

In [ ]:
# Responda a pergunta com base apenas no contexto

chain_rag = prompt_rag | llm | StrOutputParser()

pergunta = "Que dia é hoje? Retorne a data em formato dd/mm/yyyy"

res = chain_rag.invoke({"pergunta": pergunta, "contexto": contexto})
res

Dica: Se a pergunta for muito vaga e referente a algo que o modelo possa saber (por exemplo, algo muito conhecido) pele pode alucinar um pouco na resposta as vezes. Para evitar alucinar, pode colocar no template o seguinte: "Responda a pergunta com base apenas no contexto". Lembre-se que você pode sempre tentar aperfeiçoar o prompt para melhorar os resultados



## RAG - Explorando mais

Vamos pegar um exemplo mais concreto. Vamos supor que queremos usar o LLMs para responder duvidas de um documento ou uma planilha que contém informações sobre um empresa. Esses dados são privados e portanto os modelos não tem como saber (e mesmo que fossem públicos, bom lembrar que a precisão seria comprometida, sem falar que dados muitos recentes não estariam inclusos)

In [ ]:
prompt_rag

In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

contexto = """Faturamento trimestral:
1º: R$42476,40
2º: R$46212,97
3º: R$41324,56
4º: R$56430,24"""

#pergunta = "Qual é o faturamento do segundo trimestre?"
pergunta = "Qual trimestre teve o maior faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

## Depuração / Debugging

Assim como na construção de qualquer tipo de software, em algum momento você precisará depurar ao construir com LLMs. Isso porque uma chamada de modelo falhará ou a saída do modelo será malformatada, ou haverá algumas chamadas de modelo aninhadas e não ficará claro onde ao longo do caminho uma saída incorreta foi criada.

Há três métodos principais para depuração:

Modo Debug: Isso adiciona instruções de registro para TODOS os eventos em sua cadeia.
Modo Verbose: Isso adiciona instruções de impressão para eventos "importantes" em sua cadeia.
Rastreamento com LangSmith: Isso registra eventos no LangSmith para permitir a visualização lá.

Veremos o método com LangSmith mais tarde.

Agora, vamos aprender como fazer isso ativando o modo debug

https://python.langchain.com/v0.2/docs/how_to/debugging/

In [ ]:
from langchain.globals import set_debug
set_debug(True)

Definir a depuração = True com que todos os componentes do LangChain com suporte a callback (chains, modelos, agentes, tools, retrievers, etc.) imprimam as entradas que recebem e as saídas que geram. Esta é a configuração mais detalhada e registrará totalmente as entradas e saídas brutas

In [ ]:
pergunta = "Qual trimestre teve o menor faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

In [ ]:
set_debug(False)

Alternativamente, você pode usar um serviço externo como o LangSmith para rastrear melhor: https://smith.langchain.com/ LangSmith irá capturar os detalhes internos em cada etapa